In [13]:
import laspy
from laspy.compression import LazBackend
import numpy as np
import open3d as o3d
from matplotlib import colors as mcolors
from copy import copy
from collections import Counter

# 1. Read LAZ file
las = laspy.read(
    "/kaggle/input/dhruvs-pc2/PC-Trellis_39.4ft_Nadir-UltraHigh-micro.laz",
    laz_backend=LazBackend.Laszip
)

# 2. Extract XYZ and normalize RGB
xyz = np.vstack((las.x, las.y, las.z)).T
rgb = np.vstack((las.red, las.green, las.blue)).T.astype(np.float64) / 65535.0

# 3. HSV filtering
hsv = mcolors.rgb_to_hsv(rgb)
h, s, v = hsv[:, 0], hsv[:, 1], hsv[:, 2]
mask = (
    ((h >= 0.96) | (h <= 0.10)) &
    (s >= 0.20) & (s <= 0.60) &
    (v >= 0.60) & (v <= 1.00)
)

apple_xyz = xyz[mask]
print(f" Apple-colored points detected: {len(apple_xyz)}")

# 4. DBSCAN clustering
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(apple_xyz)

labels = np.array(
    pcd.cluster_dbscan(eps=0.025, min_points=30, print_progress=False)
)

# Cluster info
label_counts = Counter(labels)
print("\n Cluster sizes (label: count):")
for lbl, count in label_counts.items():
    if lbl != -1:
        print(f"  Apple {lbl}: {count} points")
print(f" Noise points: {label_counts.get(-1, 0)}")

# 5. Improved apple count estimation
MIN_APPLE_POINTS = 80
MAX_APPLE_POINTS = 400
MAX_APPLES_PER_CLUSTER = 5

cluster_sizes = [count for lbl, count in label_counts.items() if lbl != -1]
normal_clusters = [size for size in cluster_sizes if MIN_APPLE_POINTS <= size <= MAX_APPLE_POINTS]

if normal_clusters:
    avg_apple_size = np.median(normal_clusters)
else:
    avg_apple_size = 200  # fallback if no normal clusters

estimated_total_apples = 0
print(f"\n📏 Using median apple size: {avg_apple_size:.1f} points")

for lbl, size in label_counts.items():
    if lbl == -1:
        continue
    if size <= MAX_APPLE_POINTS:
        estimated_total_apples += 1
    else:
        est = int(round(size / avg_apple_size))
        est = min(est, MAX_APPLES_PER_CLUSTER)
        estimated_total_apples += est
        print(f" Cluster {lbl} = {size} pts → estimated {est} apples (capped)")

print(f"\n Adjusted total estimated apples: {estimated_total_apples}")

# 6. Save clustered apple points to LAS
apple_points = apple_xyz[labels != -1]
header_points = copy(las.header)
header_points.point_count = len(apple_points)

points_las = laspy.LasData(header_points)
points_las.x = apple_points[:, 0]
points_las.y = apple_points[:, 1]
points_las.z = apple_points[:, 2]
points_las.red   = np.full(len(apple_points), 65535, dtype=np.uint16)
points_las.green = np.zeros(len(apple_points), dtype=np.uint16)
points_las.blue  = np.zeros(len(apple_points), dtype=np.uint16)

points_las.write("apple_points_filtered.las")
print(f"\n Saved apple_points_filtered.las with {len(apple_points)} clustered red points.")

# 7. Bounding boxes for each cluster
all_corners = []
valid_cluster_ids = [lbl for lbl in set(labels) if lbl != -1]
for lbl in valid_cluster_ids:
    idxs = np.where(labels == lbl)[0]
    cluster = pcd.select_by_index(idxs)
    obb = cluster.get_oriented_bounding_box()
    corners = np.asarray(obb.get_box_points())
    all_corners.append(corners)

all_corners = np.vstack(all_corners)

header_boxes = copy(las.header)
header_boxes.point_count = len(all_corners)

box_las = laspy.LasData(header_boxes)
box_las.x = all_corners[:, 0]
box_las.y = all_corners[:, 1]
box_las.z = all_corners[:, 2]
box_las.red   = np.full(len(all_corners), 65535, dtype=np.uint16)
box_las.green = np.zeros(len(all_corners), dtype=np.uint16)
box_las.blue  = np.zeros(len(all_corners), dtype=np.uint16)

box_las.write("apple_bboxes.las")
print(f" Saved apple_bboxes.las with {len(all_corners)} red box corners.")


✅ Apple-colored points detected: 12657

📊 Cluster sizes (label: count):
  Apple 0: 221 points
  Apple 1: 32 points
  Apple 2: 124 points
  Apple 3: 77 points
  Apple 4: 126 points
  Apple 5: 299 points
  Apple 6: 191 points
  Apple 7: 176 points
  Apple 8: 718 points
  Apple 9: 2766 points
  Apple 10: 5320 points
  Apple 11: 152 points
  Apple 12: 1871 points
  Apple 13: 131 points
  Apple 14: 106 points
  Apple 15: 96 points
  Apple 16: 29 points
🗑 Noise points: 222

📏 Using median apple size: 141.5 points
🔴 Cluster 8 = 718 pts → estimated 5 apples (capped)
🔴 Cluster 9 = 2766 pts → estimated 5 apples (capped)
🔴 Cluster 10 = 5320 pts → estimated 5 apples (capped)
🔴 Cluster 12 = 1871 pts → estimated 5 apples (capped)

✅ Adjusted total estimated apples: 33

📍 Saved apple_points_filtered.las with 12435 clustered red points.
📦 Saved apple_bboxes.las with 136 red box corners.
